In [1]:
import dash
import dash_renderer
import plotly.graph_objs as go # wichtig um Layout anzupassen wie Titel, x und y-Achsenbeschriftung etc. und für verschiedene Graphen wie scatterplot 
import dash_html_components as html
import dash_core_components as dcc

from dash.dependencies import Output, Input 

import pandas as pd

from datetime import datetime as dt # für Datepicker

In [2]:
# Datenvorverarbeitung
data = pd.read_csv("data_01.csv")
data_verzoegert = data[data["deviation cause"]!= 0]
data_ver_net= data_verzoegert.groupby(["deviation cause"])["Net Value"].sum() #["Net Value", "deviation indicator", "deviation cause text"]
data_ver_net
data_verzoegert

,Unnamed: 0,Document Date,Year/Month,Year,Month,supplier delivery date,delivery date,Company Code,Country,Purchasing Doc.,...,ORDERED Quantity,Delivered QTY,open quantity,deviation indicator,deviation cause,deviation cause text,Net Value,Local Currency,Counter,Delivery deviation in days
0,0,2019-02-28,2019/2,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,...,10.0,7.0,3,in time,3,under-delivery,6720.00,EUR,1,0.0
1,1,2019-02-28,2019/2,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,...,10.0,7.0,3,in time,3,under-delivery,18162.00,EUR,1,0.0
2,2,2019-02-28,2019/2,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,...,10.0,7.0,3,in time,3,under-delivery,5710.00,EUR,1,0.0
3,3,2019-02-28,2019/2,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,...,10.0,7.0,3,in time,3,under-delivery,29890.00,EUR,1,0.0
4,4,2019-02-28,2019/2,2019,2,2019-03-07,2019-03-07,52,AT,8207406178,...,13.0,10.0,3,in time,3,under-delivery,9646.00,EUR,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,12028,2020-01-04,2020/1,2020,1,2020-01-11,2020-01-15,53,GB,8207890736,...,8.0,8.0,0,late: < 5 days,7,delivery deviation - too late,10165.44,EUR,1,4.0
12036,12036,2020-01-21,2020/1,2020,1,2020-01-28,2020-02-01,53,GB,8207914670,...,6.0,6.0,0,late: < 5 days,7,delivery deviation - too late,3479.52,EUR,1,4.0
12037,12037,2020-01-21,2020/1,2020,1,2020-01-28,2020-02-01,53,GB,8207914671,...,4.0,4.0,0,late: < 5 days,7,delivery deviation - too late,5297.32,EUR,1,4.0
12044,12044,2020-02-01,2020/2,2020,2,2020-02-08,2020-02-12,53,GB,8207935452,...,8.0,8.0,0,late: < 5 days,7,delivery deviation - too late,10165.44,EUR,1,4.0


In [3]:
data_verzoegert_weiter = data_verzoegert 
data_verzoegert = data_verzoegert.drop_duplicates(subset = "Purchasing Doc.") # etwas unsauber aber funktioniert um nur die Anzahl an Bestellungen zu bekommen
temp = sorted(data_verzoegert["deviation cause"].unique())
x = {}
for i in temp:
    x.update({int(i):len(data_verzoegert[data_verzoegert["deviation cause"]==i])}) # muss noch angepasst werden auf unique Bestellungen

x.values()

dict_values([13, 3, 30, 1, 9, 99, 3, 1, 6])

In [4]:
data3 = data_ver_net.to_frame()
data3.loc[:,"anzahl"] = x.values()#.loc[:,'numbers of deviation'] = x.values()

In [5]:
data3.index
# Ende der Datenvorbereitung für ersten Graph bei 2.3

Int64Index([1, 2, 3, 4, 5, 7, 8, 9, 10], dtype='int64', name='deviation cause')

# Datenvorbereitung zweiter Graph 2.3.2
## soll Höhe der Verzögerung darstellen bei den zu spät Lieferungen erstmal als scatterplot

In [6]:
data_verzoegert_weiter = data_verzoegert_weiter[data_verzoegert_weiter['deviation cause']==7].drop_duplicates(subset="Purchasing Doc.") # Annahme: Lieferung kommt immer zusammen
data_3_2_1 = data_verzoegert_weiter.groupby("deviation indicator").count()
data_3_2_2 = data_verzoegert_weiter.groupby("deviation indicator")["Net Value"].sum()
data_3_2_1.iloc[:,0].to_frame()

,Unnamed: 0
deviation indicator,
late: 5 to 10 days,35
late: < 5 days,59
late: > 10 days,5


In [7]:
app = dash.Dash()

In [8]:
colors = { # Farben können an den Styleguide angepasst werden und werden für alle Diagramme übernommen
    'text1' : '#ff0000', # Farbe für normale Schrift
    'text2' : '#0B610B', # Variation der Schriftfarbe
    # Farbe für Hintergründe des Plots
    'plot_bgcolor' : '#000000', # um papercolor herum außerhalb des plots
    'paper_bgcolor' : '#888888',# Farbe für Hintergründe des Paperbackground (vor plot background)
    'color_text_in_plots': '#ffffff' # Farbe für die Texte im Plot
}

# Dashboard Seite 2 Aufgabe 3:
  ##  Gesamtsumme Ordered Spend & Anzahl Bestellungen der abweichenden Lieferungen nach Abweichungsgründen (deviation cause) & Höhe der Verzögerung (deviation indicator) 

In [9]:
app.layout = html.Div([
    html.H1(children = 'Lieferanten',
            style = {
                'textAlign' : 'center',
                'color' : colors['text1']
            }
    
    
    ),
    html.Br(),
    html.Div(children = 'Dashboard for suppliers',
            style = {
                'textAlign' : 'center',
                'color' : colors['text1']
            }),
    html.Div([
        html.Div([
            html.Label('Choose between "Net Value" and "Number of orders"'),
            dcc.Dropdown(
                id = 'dropdown3',
                options = [
                    {'label': 'Net Value', 'value': 'net_value'}, # value ist eine Variable, die weitergegeben werden kann --> backend; label ist das, was am Ende dasteht --> frontend 
                    {'label': 'Number of orders', 'value': 'number_of_orders'},
                ],
                    value = 'net_value', # Startvariable
                    multi = True # mehrere können gleichzeitig gewählt werden

                        # weitere Möglichkeiten: placeholder = <text als String> --> keine Anfangsvariable und Label kann gespart werden
                        # disabled = True --> nichts kann ausgewählt werden; Bsp. muss erst ein anderer Filter ausgewählt werden (auch möglich bei options)
            )
        ]),
        dcc.Graph( # bar chart aus Aufgabe 2 Stichpunkt 3
            id = "bar3",
            figure = 
            {
                'data': 
                [
                    {
                        'x': data3.index,
                        'y': data3["Net Value"],
                        'type': 'bar',
                        'name': 'deviation causes and net value'
                    },
                    {
                        'x': data3.index,
                        'y': data3["anzahl"],
                        'type': 'bar',
                        'name': 'deviation causes and number of orders'
                    }
                    # funktionert aber um es schöner in einem Diagramm zu machen zwei y-Achsen (links und rechts) verwenden
                    # Skalierung schlecht
                ],
                # brauchen noch einen Styleguide für Fabren etc.
                'layout' :
                {   
                    'plot_bgcolor' : colors['plot_bgcolor'], # um papercolor herum außerhalb des plots
                    'paper_bgcolor' : colors['paper_bgcolor'], # das was direkt um den plot rum isst als Hintergrund
                    'font': {
                        'color': colors['color_text_in_plots'],
                        'fontsize': '15px'
                    },
                    'title':'Ordered spend and numbers of dissenting orders according to deviation causes'
                }

            }
        ),
    ]),
    # dcc.Graph(
    #     id = "scatter3",
    #     figure= {
    #         'data':
    #         [
                
    #         ]




    #     }

    # ),
    html.Br(),
    html.Label("ab hier folgen nur Tests"),
    html.Br(),
    html.Br(),
    html.Label('Testing slider maybe use for months'),
    dcc.RangeSlider(
        min = 1,
        max = 12,
        value = [0,6],
        step = 1,
        marks = {i: i for i in range(1,13,1)}

    ),
    # weitere Möglichkeiten von dcc: Inputbox, Textarea, Checklist (ähnlich verwendbar wie Dropdown), RadioItems, Markdown (commonmark.org) --> geht das fürs numeric point chart?
    # dash.plot.ly 
    # Möglichkeiten von html: Button,  

    dcc.DatePickerSingle(
        id = 'test-date-picker',
        date = dt(2019,10,12)


    ),
    # ähnlich dazu DatePickerRange mit start und enddate
    html.Div([
        dcc.Input(id = 'graph3i', value = 'net_value', type = 'text')
    
    ])
])

In [10]:
# @app.callback(
#     Output('bar3','figure'), #id des zu ändernden Graphen reinschreiben, was soll geändert werden
#     [Input('graph3i', 'value')] # von wem soll was geändert werden
# )
# def update_fig3(input_value):
#     pass

In [11]:
if __name__ =='__main__':
    app.run_server(port=8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Aug/2021 16:36:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 16:36:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 16:36:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2021 16:36:48] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
